In [1]:
import logging

logging.basicConfig(level=logging.INFO)


In [2]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss

# from pennylane import NesterovMomentumOptimizer

from qcc.ml.data import Data, BinaryData
from qcc.ml.optimize import Optimizer
from qcc.ml.data import ImageTransform, ImageTransform1D, ClassicalImageTransform
from qcc.experiment import Experiment
from qcc.ml.model import Model

from pathlib import Path
from qcc.file import new_dir

# from qcc.quantum.pennylane.ansatz import MQCCOptimized as Module
# from qcc.ml.cnn import ConvolutionalNeuralNetwork as Module
from qcc.ml.hybrid_cnn import MQCCHybrid as Module

# from qcc.quantum.pennylane.ansatz.basic import BasicFiltering6
from qcc.quantum.pennylane.pyramid import Pyramid
from qcc.quantum.pennylane.c2q import ConvolutionAngleFilter, ConvolutionComplexAngleFilter, ConvolutionFilter
from qcc.quantum.pennylane.local import define_filter


In [3]:
# Meta parameters
name = "sdfljhvosdv"
filename = Path(f"results/{name}")
num_trials = 1
silent = False
is_quantum = False

# Ansatz parameters
dims = (16, 16, 1)
num_layers = 4
module_options = {
    "U_filter": ConvolutionAngleFilter,
    # "pre_op": True,
    "num_features": 4,
    "U_fully_connected": ConvolutionAngleFilter,
    # "pooling": True,
    # # "filter_shape": (2, 2, 2),
    # "q2c_method": "parity"
    # "bias": True
}


In [4]:
filename = new_dir(filename, overwrite=True)
filename = filename / name

if is_quantum:
    module = Module.from_dims
else:
    module = Module

# Create module
module = module(
    dims,
    num_layers=num_layers,
    **module_options,
)
data = BinaryData(
    FashionMNIST,
    ImageTransform(dims) if is_quantum else ClassicalImageTransform(dims),
    batch_size=(8,8),
)
optimizer = Optimizer(Adam)
loss = CrossEntropyLoss()
model = Model.with_logging(module, data, optimizer, loss, epoch=8)

# Log important values
model.logger.info(f"Circuit ID: {name}")
model.logger.info(f"{module=}")
model.logger.info(f"{data=}")
model.logger.info(f"{optimizer=}")
model.logger.info(f"{loss=}")

model.logger.info(f"{num_trials=}")
model.logger.info(f"{dims=}")
model.logger.info(f"{num_layers=}")
# model.logger.info(f"{1=}")
model.logger.info(f"{module_options=}")

# Save circuit drawing
if is_quantum:
    filename = filename.with_stem(f"{name}_circuit")
    module.draw(filename=filename, decompose=True)


INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 22
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 88
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 8
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 29
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 7
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 21
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 6
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 15
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 9
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 19
2023-10-26 23:19:14,462: (mqcchybrid) Circuit ID: sdfljhvosdv
2023-10-26 23:19:14,463: (mqcchybrid) module=MQCCHybrid(
  (0): MQCCLayer(
    (mqcc): MQCC()
  )
  (1): ReLU()
  (2): MQCCLayer(
    (mqcc): MQCC()
  )
  (3): ReLU()
  (4): MQCCLayer(
    (mqcc): MQCC()
  )
  (5): ReLU()
  (6): MQCCLayer(
    (mqcc): MQCC()
  )
  (7): ReLU()
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): FullyConnectedLayer(
    (mqcc): FullyConnected()
  )
)
2023-10-26 23:19:14,463: (m

In [5]:
# Run experiment
results_schema = ["accuracy", "training_time", "testing_time"]
experiment = Experiment(model, num_trials, results_schema)
# experiment.partial(silent=silent)
results = experiment(filename=filename)


2023-10-26 23:19:14,506: (mqcchybrid) Number of Parameters: 29
/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/src/qcc/quantum/quantum.py:125: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3618.)
  data = data.reshape(new_dims[::-1]).T
2023-10-26 23:20:24,741: (mqcchybrid) (Epoch 1) Training took 70.00171 sec
2023-10-26 23:20:33,333: (mqcchybrid) (Epoch 1) Testing took: 8.56444 sec
2023-10-26 23:20:33,333: (mqcchybrid) (Epoch 1) Accuracy: 87.650%


KeyboardInterrupt: 

In [ ]:
# Print accuracy results
metrics = ("median", "mean", "max", "min", "std")
for name in results.columns:
    col = results[name]
    msg = (f"{metric}={getattr(col, metric)()}" for metric in metrics)
    msg = ", ".join(msg)
    msg = f"{name}: {msg}"
    model.logger.info(msg)

# Save aggregated loss history figure
display(experiment.draw(filename))
